In [134]:
from sklearn.model_selection import GridSearchCV
!pip install gensim
from gensim.models import Word2Vec
import pandas as pd
from nltk.stem import PorterStemmer
from collections import Counter
import numpy as np
from autocorrect import Speller
import re
import os
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

nltk.download("stopwords")
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
pd
lemmatizer = WordNetLemmatizer()
Spell = Speller()
stemmer = PorterStemmer()
folder_path = "./p00_tweets/"
files = ['processedPositive.csv', 'processedNeutral.csv', 'processedNegative.csv']
stop_words = set(stopwords.words("english"))

Looking in indexes: https://pypi.python.org/simple


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rezzahra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/rezzahra/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rezzahra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/rezzahra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [135]:
def clean_text(text):
    # Convert to lowercase, remove numbers, quotes, and strip leading/trailing spaces
    text = re.sub(r"\d+", "", text.lower()).replace("'", "").replace('"', "")
    # Add newlines where a comma is not followed by a space
    text = re.sub(r",(?=\S)", "\n", text)
    # Split by lines and filter out short lines (less than 2 words)
    return [line.strip() for line in text.split("\n") if len(line.split()) >= 2]
    
def normalize_repeated_letters_and_remove_non_alpha_char(text):
    # Replace 3+ repeated letters with a single occurrence
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # e.g., "happyyyy" -> "happy"
    text = re.sub(r'[^a-zA-Z\s]', '', text)   # Remove non-alphabetic characters
    return text
    
negations = {"not", "no", "nor", "never"}

def remove_stop_words(words):
    cleaned_list = [word for word in words if word not in stop_words or word in negations]
    return cleaned_list
    
def count_words(tweet):
    return Counter(tweet)

def stemmatization(words):
    stem_list = [stemmer.stem(word) for word in words]
    return stem_list

def sentiment_to_numbers(text):
    if text == "positive":
        return 4
    elif text == "negative":
        return 0
    else:
        return 2

def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjective
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verb
        return wordnet.VERB
    elif tag.startswith('N'):  # Noun
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_pos(words):
    # words = words.apply(word_tokenize)
    tags = nltk.pos_tag(words)
    lem_list = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tags]
    return lem_list

In [136]:
tweets = {}
for file in files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as f:
        raw_text = f.read()
    # Clean the text and remove duplicates
    text = clean_text(raw_text)
    processed_text = list(set(text))
    # Save processed tweets in the dictionary
    key = file.replace("processed", "").replace(".csv", "").lower()
    tweets[key] = processed_text

    

# Output: A dictionary with cleaned and filtered tweets
data = []
for sentiment, tweet_list in tweets.items():
    for tweet in tweet_list:
        data.append((tweet, sentiment))

# Create a DataFrame
df = pd.DataFrame(data, columns=["text", "sentiment"])
#top  similar tweets
features = df.text

word_token_sent=[]
for i in range(0,len(features)):
    word_token_sent.append(word_tokenize(features[i]))
filtered_sentence = []
for w in word_token_sent[1]:
    if w not in stop_words:
        filtered_sentence.append(w)
processed_features = []
for sentence in range(0, len(features)):
    processed_feature = re.sub(r'\@.*?\s','', str(features[sentence])) #removing mentions
    processed_feature = re.sub(r'\W', ' ', str(processed_feature)) #special character removal
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature) #single charracter
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I) #multi space issues handled
    processed_feature = re.sub(r'^b\s+', '', processed_feature) #removing prefix b

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)
documents = set(processed_features)

similarity_index=cosine_similarity(tfidf_matrix[6:7],tfidf_matrix)
print(processed_features[6]) # input tweet
similarity_index=similarity_index.reshape(-1)
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(tfidf_matrix.shape)
n=10
print (similarity_index[np.argsort(similarity_index)[-n:]])
res = sorted(range(len(similarity_index)), key = lambda sub: similarity_index[sub])[-n:]   
# printing result 
print("top 10 similar tweets : " + str(res))
for i in res:
    print(processed_features[i])
    print('\n')

new video stream montage we finally unboxed spectrum knife hope you like it here 
(2626, 6286)
[0.14594048 0.14887815 0.15555018 0.16214686 0.16240844 0.16497485
 0.17987906 0.2062324  0.22971579 1.        ]
top 10 similar tweets : [2118, 1816, 1095, 1507, 387, 553, 2445, 1459, 1074, 6]
aww eunji didnt see win today sad but congratulations to winner 


scout photobombed my bluebells photo sad


flash after supreme courts order fir registered against up minister over gang rape sexual harassment


rehearsing for and more in our guwahati epaper 


indeed and suspect you will be one of the few who would get it happy


funny happy


friendly fire to wings tour unhappy


how is going against the wave of transparency in


yogendra yadav on the future of india under why get it wrong and more 


new video stream montage we finally unboxed spectrum knife hope you like it here 




In [137]:
def preprocess_text(text, technique):
    if "custom preprocessing" in technique:
        text = text.apply(normalize_repeated_letters_and_remove_non_alpha_char)
    text = text.apply(word_tokenize)
    if "custom preprocessing" in technique:
        text = text.apply(remove_stop_words)
    # print(f"preprocessText: {text}")
    if technique == "tokenization":
        return text.apply(lambda x: ' '.join(x))
    elif technique == "stemming":
        return text.apply(lambda x: ' '.join(stemmatization(x)))
    elif technique == "lemmatization":
        return text.apply(lambda x: ' '.join(lemmatize_pos(x)))
    elif technique == "stemming + misspellings" or technique == "stemming + misspellings + custom preprocessing":
        stemmed = text.apply(lambda x: stemmatization(x))
        return text.apply(lambda x: ' '.join([Spell(word) for word in x]))
    elif technique == "lemmatization + misspellings" or technique == "lemmatization + misspellings + custom preprocessing":
        lemmatized = text.apply(lambda x: lemmatize_pos(x))
        return lemmatized.apply(lambda x: ' '.join([Spell(word) for word in x]))
    else:
        return text.apply(lambda x: ' '.join(x))

def evaluate_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    logreg = LogisticRegression(max_iter=5000,random_state=42)  # saga is better for sparse data
    param_grid = [{'C': 10**np.linspace(-3, 3, 20), 'solver': ['lbfgs', 'saga', 'liblinear']}]
    logreg_cv=GridSearchCV(logreg,param_grid, cv=10, scoring='accuracy', 
                             refit=True)
    logreg_cv.fit(X_train,y_train)
    print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
    y_pred = logreg_cv.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(classification_report(y_test,y_pred))
    print(f"accuracy_score function - Accuracy: {accuracy}")
    return accuracy

    

In [138]:



def train_word2vec(sentences, vector_size=100, window=5, min_count=2):
    model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    return model

def get_average_word2vec(sentences, model, vector_size):
    vectors = []
    for sentence in sentences:
        words = sentence.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            vectors.append(sum(word_vectors) / len(word_vectors))
        else:
            vectors.append([0] * vector_size)
    return np.array(vectors)

def prepare_features(text, method):
    if method == "binary":
        vectorizer = CountVectorizer(binary=True, max_features=5000)
        return vectorizer.fit_transform(text)
    elif method == "word_counts":
        vectorizer = CountVectorizer(max_features=5000)
        return vectorizer.fit_transform(text)
    elif method == "tfidf":
        vectorizer = TfidfVectorizer(max_features=5000)
        return vectorizer.fit_transform(text)
    elif method == "word2vec":
        tokenized_text = [sentence.split() for sentence in text]
        w2v_model = train_word2vec(tokenized_text)
        return get_average_word2vec(text, w2v_model, vector_size=100)

def test_combinations(df):
    results = []
    techniques = ["tokenization", "stemming", "lemmatization", "stemming + misspellings", "lemmatization + misspellings","lemmatization + misspellings + custom preprocessing","stemming + misspellings + custom preprocessing"]
    methods = ["binary", "word_counts", "tfidf", "word2vec"]

    for technique in techniques:
        for method in methods:
            print(f"Testing: {technique} + {method}")
            processed_text = preprocess_text(df["text"], technique)
            X = prepare_features(processed_text, method)
            y = df['sentiment'].apply(sentiment_to_numbers)
            accuracy = evaluate_model(X, y)
            results.append((technique, method, accuracy))
            print(f"Accuracy: {accuracy:.4f}")

    return results


In [139]:
# Run the Test

results = test_combinations(df)

# # Display Results
results_df = pd.DataFrame(results, columns=["Preprocessing Technique", "Feature Preparation", "Accuracy"])
print(results_df)

Testing: tokenization + binary


KeyboardInterrupt: 